In [179]:
import json
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

In [180]:
with open('dataset.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)
print(df.head())

                                               title  \
0  Cá nhân hóa trải nghiệm bằng công nghệ tiên ti...   
1                       Microsoft ra mắt Office 2024   
2  Những thành tựu khó đạt được nhất trong Black ...   
3  Tựa game "đắt" nhất trên Steam, mua đủ hết nội...   
4  CEO Game Science trả lời hài hước về khả năng ...   

                                            subtitle    category          time  
0  Chính thức hoạt động từ cuối tháng 7/2024, Ngâ...  Apps/Games  5 ngày trước  
1  Office 2024 đã chính thức ra mắt với hàng loạt...  Apps/Games  5 ngày trước  
2  Nếu như không có sự trợ giúp từ Internet, các ...  Apps/Games  8 ngày trước  
3  Thế nhưng đa phần các game thủ Steam đều chẳng...  Apps/Games  8 ngày trước  
4          Trong lúc chờ, bạn hãy chơi game khác đi!  Apps/Games  9 ngày trước  


In [181]:
tmp_label = ['Apps/Games', 'Internet', 'Mobile', 'Tin ICT', 'Đồ chơi số'] #, 'Khám phá'
df = df[df['category'].isin(tmp_label)]
df['text'] = df['title'] + " " + df['subtitle']
df = df[['text', 'category']]
print(df.head())

                                                text    category
0  Cá nhân hóa trải nghiệm bằng công nghệ tiên ti...  Apps/Games
1  Microsoft ra mắt Office 2024 Office 2024 đã ch...  Apps/Games
2  Những thành tựu khó đạt được nhất trong Black ...  Apps/Games
3  Tựa game "đắt" nhất trên Steam, mua đủ hết nội...  Apps/Games
4  CEO Game Science trả lời hài hước về khả năng ...  Apps/Games


In [182]:
# nltk.download('stopwords')
# stop_words = set(stopwords.words('vietnamese'))
stop_words = []

In [183]:
def clean_text(text):
    # Chuyển văn bản về chữ thường
    text = text.lower()
    
    # Loại bỏ ký tự đặc biệt và dấu câu
    text = re.sub(r'\W', ' ', text)
    
    # Loại bỏ các số
    text = re.sub(r'\d+', '', text)
    
    # Loại bỏ các từ dừng (stopwords)
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    return text

In [184]:
df['text'] = df['text'].apply(clean_text)
print(df.head())

                                                text    category
0  cá nhân hóa trải nghiệm bằng công nghệ tiên ti...  Apps/Games
1  microsoft ra mắt office office đã chính thức r...  Apps/Games
2  những thành tựu khó đạt được nhất trong black ...  Apps/Games
3  tựa game đắt nhất trên steam mua đủ hết nội du...  Apps/Games
4  ceo game science trả lời hài hước về khả năng ...  Apps/Games


In [185]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9685 entries, 0 to 11990
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      9685 non-null   object
 1   category  9685 non-null   object
dtypes: object(2)
memory usage: 227.0+ KB


In [186]:
df.describe()

,text,category
count,9685,9685
unique,9503,5
top,apple công bố doanh thu khủng trong quý không ...,Tin ICT
freq,3,2090


In [187]:
# Chuyển đổi văn bản thành ma trận TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)  # Giới hạn số lượng đặc trưng
X = vectorizer.fit_transform(df['text']).toarray()

# Chuyển nhãn (category) thành dạng số
le = LabelEncoder()
y = le.fit_transform(df['category'])

In [188]:
print(np.unique(df['category']))
print(X.shape)
print(y.shape)
print()
print(df['category'].value_counts())

['Apps/Games' 'Internet' 'Mobile' 'Tin ICT' 'Đồ chơi số']
(9685, 1000)
(9685,)

category
Tin ICT       2090
Mobile        2008
Đồ chơi số    1952
Internet      1934
Apps/Games    1701
Name: count, dtype: int64


# Split Dataset

In [189]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LosisticRegression

In [ ]:
model_logistic_regression = LogisticRegression()
model_logistic_regression.fit(X_train, y_train)

# Predict

In [172]:
y_pred = model_logistic_regression.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print(classification_report(y_test, y_pred, target_names=le.classes_))

Accuracy: 0.6871450696954052
              precision    recall  f1-score   support

  Apps/Games       0.70      0.74      0.72       351
    Internet       0.60      0.58      0.59       407
      Mobile       0.85      0.82      0.84       401
     Tin ICT       0.54      0.54      0.54       411
  Đồ chơi số       0.76      0.76      0.76       367

    accuracy                           0.69      1937
   macro avg       0.69      0.69      0.69      1937
weighted avg       0.69      0.69      0.69      1937



# RandomForest Classifier

In [173]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

RandomForestClassifier()

In [174]:
y_pred_rf = rf.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.6623644811564274
              precision    recall  f1-score   support

           0       0.71      0.73      0.72       351
           1       0.59      0.54      0.56       407
           2       0.79      0.86      0.82       401
           3       0.50      0.48      0.49       411
           4       0.70      0.73      0.72       367

    accuracy                           0.66      1937
   macro avg       0.66      0.67      0.66      1937
weighted avg       0.66      0.66      0.66      1937



# SVM

In [ ]:
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

In [ ]:
y_pred_svm = svm.predict(X_test)

print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))

# Bayes

In [ ]:
nb = MultinomialNB()
nb.fit(X_train, y_train)

In [ ]:
y_pred_nb = nb.predict(X_test)

print("Naive Bayes Accuracy:", accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))

# KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

In [ ]:
y_pred_knn = knn.predict(X_test)

print("KNN Accuracy:", accuracy_score(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn))

# -------

In [175]:
new_text = "Apple ra mắt sản phẩm mới MacBook Pro 2024 với thiết kế mỏng hơn"
new_text_clean = clean_text(new_text)

# Chuyển đổi thành vector TF-IDF
new_text_vector = vectorizer.transform([new_text_clean])

predicted_category = le.inverse_transform(model_logistic_regression.predict(new_text_vector))

print(f'Predicted Category: {predicted_category[0]}')


Predicted Category: Đồ chơi số
